# 파이썬 플라스크 API 서버 테스트 #

In [199]:
'''
CREATE TABLE guardian (
  num int(11) NOT NULL AUTO_INCREMENT,
  uid varchar(30) NOT NULL,
  name varchar(50) NOT NULL,
  tel varchar(12) NOT NULL,
  birthday varchar(7) NOT NULL,
  pid varchar(255) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE guardian (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  uid varchar(30) NOT NULL,\n  name varchar(50) NOT NULL,\n  tel varchar(12) NOT NULL,\n  birthday varchar(7) NOT NULL,\n  pid varchar(255) NOT NULL,\n  rdate datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [200]:
!sudo pip3 install pymysql

In [201]:
!pip install flask-ngrok

In [202]:
!sudo pip3 install flask_restful

In [203]:
import pymysql
import configparser
from flask import Flask
from datetime import datetime
from google.colab import drive
from flask_restful import reqparse
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

In [204]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [205]:
CNF_INI = 'drive/My Drive/development/config/openapi.ini'
ERR_URI = 'drive/My Drive/development/test/error.log'

In [206]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']

In [207]:
def exec_select(conn,uid):
  with conn.cursor() as cursor:
    query = get_select(uid)
    cursor.execute(query)
    for row in cursor:
      data = row.get('cnt')
      
  return data

In [208]:
def exec_insert(conn,uid,name,tel,birthday,pid):
  query = get_insert(uid,name,tel,birthday,pid)
  with conn.cursor() as cursor:
    cursor.execute(query)
  conn.commit()

In [209]:
def get_select(uid):
  query = 'select \
  count(*) as cnt \
  from guardian \
  where uid=\'{}\''.format(uid)
  print(query)
  return query

In [210]:
def get_insert(uid,name,tel,birthday,pid):
  query = 'insert into guardian (uid,name,tel,birthday,pid) \
  values (\'{}\',\'{}\',\'{}\',\'{}\',\'{}\')'.format(uid,name,tel,birthday,pid)
  print(query)
  return query

In [211]:
class blank_page(Resource):
  def get(self):
    return {'status':'200'}

In [212]:
class create_user(Resource):
  def get(self):
    #return {'status':'success'}
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('uid', type=str)
      parser.add_argument('name', type=str)
      parser.add_argument('tel', type=str)
      parser.add_argument('birthday', type=str)
      parser.add_argument('pid', type=str)
      args = parser.parse_args()
      
      uid = args['uid']
      name = args['name']
      tel = args['tel']
      birthday = args['birthday']
      pid = args['pid']

      # print(uid,name,tel,birthday,pid)
      
      conn = pymysql.connect(
          host=HOST,
          user=USER,
          password=PASSWORD,
          db=DATABASE,
          charset=CHARSET,
          port=int(PORT),
          cursorclass=pymysql.cursors.DictCursor)
      
      # print(conn)

      cnt = exec_select(conn,uid)
      
      if cnt:
        return {'StatusCode': '1000', 'Message': 'The account exists'}
      else:
        exec_insert(conn,uid,name,tel,birthday,pid)
        return {'StatusCode': '1100', 'Message': 'You have created an account. ('+uid+')'}
     
    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))

In [213]:
app = Flask(__name__)
api = Api(app)

In [ ]:
run_with_ngrok(app)
api.add_resource(blank_page,'/')
api.add_resource(create_user,'/user')
app.run()